In [4]:
from coderm.prompts import py_prompt_evolve, py_prompt
from vllm import LLM, SamplingParams

In [2]:
model = LLM("federico-staging/train_sc2_15b_evolver")

/mnt/efs/federicocassano/.env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

INFO 06-13 20:31:17 config.py:1130] Casting torch.float32 to torch.float16.


INFO 06-13 20:31:17 config.py:1151] Downcasting torch.float32 to torch.float16.


INFO 06-13 20:31:17 llm_engine.py:161] Initializing an LLM engine (v0.4.3) with config: model='federico-staging/train_sc2_15b_evolver', speculative_config=None, tokenizer='federico-staging/train_sc2_15b_evolver', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=16384, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=federico-staging/train_sc2_15b_evolver)


...

INFO 06-13 20:38:49 model_runner.py:146] Loading model weights took 29.7278 GB


INFO 06-13 20:38:52 gpu_executor.py:83] # GPU blocks: 31464, # CPU blocks: 3276


INFO 06-13 20:38:53 model_runner.py:854] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.


INFO 06-13 20:38:53 model_runner.py:858] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.


INFO 06-13 20:38:59 model_runner.py:924] Graph capturing finished in 6 secs.


In [15]:
q = "You have an empty sequence A. There are Q queries given, and you need to process them in the order they are given.\nThe queries are of the following two types:\n\n- 1 x: Append x to the end of A.\n- 2 k: Find the k-th value from the end of A. It is guaranteed that the length of A is at least k when this query is given.\n\nInput\n\nThe input is given from Standard Input in the following format:\nQ\n\\mathrm{query}_1\n\\mathrm{query}_2\n\\vdots\n\\mathrm{query}_Q\n\nEach query is in one of the following two formats:\n1 x\n\n2 k\n\nOutput\n\nPrint q lines, where q is the number of queries of the second type.\nThe i-th line should contain the answer to the i-th such query.\n\nConstraints\n\n\n- 1 \\leq Q \\leq 100\n- In the first type of query, x is an integer satisfying 1 \\leq x \\leq 10^9.\n- In the second type of query, k is a positive integer not greater than the current length of sequence A.\n\nSample Input 1\n\n5\n1 20\n1 30\n2 1\n1 40\n2 3\n\nSample Output 1\n\n30\n20\n\n\n- Initially, A is empty.\n- The first query appends 20 to the end of A, making A=(20).\n- The second query appends 30 to the end of A, making A=(20,30).\n- The answer to the third query is 30, which is the 1-st value from the end of A=(20,30).\n- The fourth query appends 40 to the end of A, making A=(20,30,40).\n- The answer to the fifth query is 20, which is the 3-rd value from the end of A=(20,30,40)."
code = "\n\n# Read the input values for the volumes of the regions\n(v1, v2, v3) = map(int, input().split())\n\n# Check if the sum of the volumes is not equal to 343\nif v1 + v2 + v3 != 343:\n    # If not, print \"No\" and exit the program\n    print('No')\n    exit()\n\n# Check if the volume of the region contained in exactly one cube is not equal to 7\nif v1 != 7:\n    # If not, print \"No\" and exit the program\n    print('No')\n    exit()\n\n# Check if the volume of the region contained in exactly two cubes is not equal to 84\nif v2 != 84:\n    # If not, print \"No\" and exit the program\n    print('No')\n    exit()\n\n# If all conditions are satisfied, print \"Yes\" and the coordinates of the cubes\nprint('Yes')\nprint(0, 0, 0, 0, 6, 0, 6, 0, 0)"
p = py_prompt(q)
#p = py_prompt_evolve(p)
print(model.generate([p], SamplingParams(temperature=0.2, max_tokens=4096))[0].outputs[0].text)

Processed prompts:   0%|                                                                                                                                                                                                                                                 | 0/1 [00:00<?, ?it/s, Generation Speed: 0.00 toks/s]

Processed prompts: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.02s/it, Generation Speed: 67.19 toks/s]

Processed prompts: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.02s/it, Generation Speed: 67.19 toks/s]



# Read the number of queries
Q = int(input())

# Initialize an empty list to store the sequence A
A = []

# Iterate through each query
for i in range(Q):
    # Read the query type and its arguments
    (a, b) = map(int, input().split())

    # If the query type is 1, append the argument to the end of the sequence A
    if a == 1:
        A.append(b)

    # If the query type is 2, print the k-th value from the end of the sequence A
    else:
        print(A[-b])
# ==== EVOLVED CODE ====

# Read the number of queries
Q = int(input())

# Initialize an empty list to store the sequence A
A = []

# Initialize an empty list to store the answers to the second type queries
ans = []

# Iterate through each query
for i in range(Q):
    # Read the query type and its arguments
    (q, x) = map(int, input().split())

    # If the query type is 1, append the argument to the end of the sequence A
    if q == 1:
        A.append(x)
    # If the query type is 2, find the k-th value from the end of A a